In [1]:
#!import "../DataModel/DataStructure"
#!import "./ApplicationMessage"

In [1]:
using System.Linq.Expressions;

# Math Functions

In [1]:
static double DivideOrDefault(double numerator, double denominator, double defaultValue = default) => Math.Abs(denominator) > double.Epsilon ? numerator / denominator : defaultValue;

In [1]:
// because the default(T) is something else than the first/last element. What about "static T GetValidElement<T>(this IList<T> array, int index)"? 
static T GetElementOrDefault<T>(this ICollection<T> array, int index)
{   
    var count = array.Count;
    if (array == null || count == 0)
        return default(T);

    return index < 0
               ? array.ElementAt(0) // should this case be removed?
               : index < count
                   ? array.ElementAt(index)
                   : array.ElementAt(count -1);
}

# Enumerable Extensions

In [1]:
public static Dictionary<TKey, TResult> ToDictionaryGrouped<TSource, TKey, TResult>(this IEnumerable<TSource> source, Func<TSource, TKey> keySelector, Func<IGrouping<TKey, TSource>, TResult> elementSelector) => source.GroupBy(keySelector).ToDictionary(g => g.Key, elementSelector);

In [1]:
public static IDataCube<TTarget> SelectToDataCube<TSource, TTarget>(this IEnumerable<TSource> source, Func<TSource, bool> whereClause, Func<TSource, TTarget> selector) => source.Where(whereClause).Select(selector).ToDataCube();

In [1]:
public static IDataCube<TTarget> SelectToDataCube<TSource, TTarget>(this IEnumerable<TSource> source, Func<TSource, TTarget> selector) => source.SelectToDataCube(x => true, selector);

In [1]:
public static double[] Prune (this IEnumerable<double> source, double precision = Precision) => source.Reverse().SkipWhile(x => Math.Abs(x) < precision).Reverse().ToArray();

# String Extensions

In [1]:
using System.Globalization;

In [1]:
public static double CheckStringForExponentialAndConvertToDouble (this string s)
{   
    if (s == null) return default;
    if (double.TryParse(s, NumberStyles.Number, CultureInfo.InvariantCulture, out var doubleValue)) return doubleValue;
    else { ApplicationMessage.Log(Error.ParsingScientificNotation, s); return 1; }
}

# Enum Extentions

In [1]:
public static bool Contains<T>(this T value, T lookingForFlag) 
    where T : struct
{
    int intValue = (int) (object) value;
    int intLookingForFlag = (int) (object) lookingForFlag;
    return ((intValue & intLookingForFlag) == intLookingForFlag);
}

# Queries

## FX

In [1]:
public static async Task<Dictionary<string, Dictionary<FxPeriod, double>>> GetExchangeRatesDictionaryAsync(this IQuerySource querySource, int year, int month)
    => (await querySource.Query<ExchangeRate>()
    .Where(x => x.Year == year - 1 && x.Month == MonthInAYear && x.FxType == FxType.Spot ||
                x.Year == year && x.Month == month)
    .ToArrayAsync())
    .ToDictionaryGrouped(x => x.Currency,
                         x => x.ToDictionary(y => (y.Year, y.Month, y.FxType) switch
                                             {
                                                 (_, _, _) when y.Year == year - 1 && y.Month == MonthInAYear && y.FxType == FxType.Spot    => FxPeriod.BeginningOfPeriod,
                                                 (_, _, _) when y.Year == year     && y.Month == month        && y.FxType == FxType.Average => FxPeriod.Average,
                                                 (_, _, _) when y.Year == year     && y.Month == month        && y.FxType == FxType.Spot    => FxPeriod.EndOfPeriod
                                             },
                                             y => y.FxToGroupCurrency));

In [1]:
public static double GetCurrencyToGroupFx(Dictionary<string, Dictionary<FxPeriod, double>> exchangeRates, string currency, FxPeriod fxPeriod, string groupCurrency)
{
    if(currency == groupCurrency)
      return 1;

    if(!exchangeRates.TryGetValue(currency, out var currencyToGroup))
      ApplicationMessage.Log(Error.ExchangeRateCurrency, currency);

    if(!currencyToGroup.TryGetValue(fxPeriod, out var currencyToGroupFx))
      ApplicationMessage.Log(Error.ExchangeRateNotFound, currency, fxPeriod.ToString());

    return currencyToGroupFx;
}

## Current and Previous Parameters

<b>TODOs:</b>
<ul>
<li> DataNodes Query: After merging with the new import storage, define IsFirstPeriod property.</li>
<li> Once Scenario is defined as a simple property (e.g. in DataNodeState), introduce concept of Priority in LoadCurrentAndPreviousParameterAsync query (see ParameterResultsEntityQueryExtensions in IfrsGeneric).</li>
</ul>

In [1]:
public static async Task<T[]> LoadParameterAsync<T>(
    this IQuerySource querySource,
    int year,
    int month,
    Expression<Func<T, bool>> filterExpression = null ) 
    where T : IWithYearAndMonth
{
    return await querySource.Query<T>()
                     .Where(x => x.Year == year && x.Month <= month || x.Year < year)
                     .Where(filterExpression?? (Expression<Func<T, bool>>)(x => true))
                     .ToArrayAsync();
}

In [1]:
public static async Task<Dictionary<string, T>> LoadCurrentParameterAsync<T> (
    this IQuerySource querySource,
    Args args,
    Func<T, string> identityExpression,
    Expression<Func<T, bool>> filterExpression = null ) 
    where T : IWithYearAndMonth
{
    return (await querySource.LoadParameterAsync<T>(args.Year, args.Month, filterExpression))
                    .GroupBy(identityExpression)
                    .Select(y => y.OrderByDescending(x => x.Year)
                                  .ThenByDescending(x => x.Month)
                                  .FirstOrDefault())
                    .ToDictionary(identityExpression);
}

In [1]:
public static async Task<Dictionary<string, Dictionary<int, T>>> LoadCurrentAndPreviousParameterAsync<T> (
    this IQuerySource querySource,
    Args args,
    Func<T, string> identityExpression,
    Expression<Func<T, bool>> filterExpression = null ) 
    where T : IWithYearAndMonth
{
    var parameters = (await querySource.LoadParameterAsync<T>(args.Year, args.Month, filterExpression)).GroupBy(identityExpression);
                                         
    var ret = new Dictionary<string, Dictionary<int, T>>();
    foreach(var p in parameters)
    {
        var inner = ret.GetOrAdd(p.Key, _ => new Dictionary<int, T>());

        var currentCandidate = p.Where(x => x.Year == args.Year).OrderByDescending(x => x.Month).FirstOrDefault();
        var previousCandidate = p.Where(x => x.Year < args.Year).OrderByDescending(x => x.Year).ThenByDescending(x => x.Month).FirstOrDefault();
    
        inner.Add(CurrentPeriod, currentCandidate != null ? currentCandidate : previousCandidate);
        inner.Add(PreviousPeriod, previousCandidate != null ? previousCandidate : currentCandidate);
    }
    return ret;
}

<a id='yield-curve'></a>
## Yield Curve

In [1]:
public static async Task<Dictionary<string, YieldCurve>> LoadLockedInYieldCurveAsync(this IQuerySource querySource, Args args, 
                                                                                                       IEnumerable<DataNodeData> dataNodes)
{
    var lockedInYieldCurveByGoc = new Dictionary<string, YieldCurve>();
    foreach (var dn in dataNodes.Where(x => x.ValuationApproach == ValuationApproaches.BBA))
    {
        var argsNew = args with {Year = dn.Year, Month = dn.Month};
        var loadedYc = (await querySource.LoadCurrentParameterAsync<YieldCurve>(argsNew, x => x.Currency, x => x.Currency == dn.ContractualCurrency));
        
        if(!loadedYc.TryGetValue(dn.ContractualCurrency, out var lockedYc))
            ApplicationMessage.Log(Error.YieldCurveNotFound, dn.ContractualCurrency, argsNew.Year.ToString(), argsNew.Month.ToString());
            
        lockedInYieldCurveByGoc[dn.DataNode] = lockedYc;
    }
    
    return lockedInYieldCurveByGoc;
}

In [1]:
public static async Task<Dictionary<string, Dictionary<int, YieldCurve>>> LoadCurrentYieldCurveAsync(this IQuerySource querySource, Args args, 
                                                                                                      IEnumerable<DataNodeData> dataNodes)
{
    var contractualCurrenciesInScope = dataNodes.Select(dn => dn.ContractualCurrency).ToHashSet();
    return (await querySource.LoadCurrentAndPreviousParameterAsync<YieldCurve>(args, 
                                                                               x => x.Currency,
                                                                               x => contractualCurrenciesInScope.Contains(x.Currency)
                                                                               ));
}

## Data Node State

In [1]:
//TODO : remove this method and clean up in queriesTest. We need a new test for LoadDataNodes.
public static async Task<Dictionary<string, DataNodeState>> LoadDataNodeStateAsync(this IQuerySource querySource, Args args)
{
    //This querySource is partitioned
    return (await querySource.LoadCurrentAndPreviousParameterAsync<DataNodeState>(args, x => x.DataNode))
                             .Where(x => x.Value[CurrentPeriod].State != State.Inactive)
                             .ToDictionary(x => x.Key, x => x.Value[CurrentPeriod]);
}

## Data Nodes

In [1]:
// TODO: the name of this method is strange, one expects DataNodes in return but instead obtains DataNodeData...
public static async Task<Dictionary<string, DataNodeData>> LoadDataNodesAsync(this IQuerySource querySource, Args args) //How do we force the user to provide here a partitioned source? IWorkspace?
{
    var dataNodeStates = await querySource.LoadCurrentAndPreviousParameterAsync<DataNodeState>(args, x => x.DataNode);
    var activeDataNodes = dataNodeStates.Where(kvp => kvp.Value[CurrentPeriod].State != State.Inactive).Select(kvp => kvp.Key);
    
    return (await querySource.Query<GroupOfContract>().Where(dn => activeDataNodes.Contains(dn.SystemName)).ToArrayAsync())
                            .ToDictionary(dn => dn.SystemName, dn => {
                                                                      var dnCurrentState = dataNodeStates[dn.SystemName][CurrentPeriod];
                                                                      var dnPreviousState = dataNodeStates[dn.SystemName][PreviousPeriod];
                                                                      return new DataNodeData(){Year = dnPreviousState.Year, 
                                                                                                  Month = dnPreviousState.Month,
                                                                                                  State = dnCurrentState.State,
                                                                                                  PreviousState = dnPreviousState.State,
                                                                                                  //from Group of Contract
                                                                                                  DataNode = dn.SystemName,
                                                                                                  ContractualCurrency  = dn.ContractualCurrency,
                                                                                                  FunctionalCurrency  = dn.FunctionalCurrency,
                                                                                                  LineOfBusiness  = dn.LineOfBusiness,
                                                                                                  ValuationApproach  = dn.ValuationApproach,
                                                                                                  OciType  = dn.OciType,
                                                                                                  Portfolio  = dn.Portfolio,
                                                                                                  AnnualCohort  = dn.AnnualCohort,
                                                                                                  LiabilityType  = dn.LiabilityType,
                                                                                                  Profitability  = dn.Profitability,
                                                                                                  Partner  = dn.Partner,
                                                                                                  IsReinsurance  = dn.GetType() == typeof(GroupOfReinsuranceContract),
                                                                                                  };
                                                                     }
                                         );
}

## Data Node Parameters

### Single data Node

In [1]:
public static async Task<Dictionary<string, Dictionary<int, SingleDataNodeParameter>>> LoadSingleDataNodeParametersAsync(this IQuerySource querySource, Args args)
{
    return await querySource.LoadCurrentAndPreviousParameterAsync<SingleDataNodeParameter>(args, x => x.DataNode);
}

### Inter data Node

In [1]:
public static async Task<Dictionary<string, Dictionary<int, HashSet<InterDataNodeParameter>>>> LoadInterDataNodeParametersAsync(this IQuerySource querySource, Args args)
{
    var identityExpressions = new Func<InterDataNodeParameter, string>[]{x => x.DataNode, x => x.LinkedDataNode,};
    var parameterArray = (await querySource.LoadParameterAsync<InterDataNodeParameter>(args.Year, args.Month));
    var parameters = identityExpressions.SelectMany(ie => parameterArray.GroupBy(ie));
                                         
    return parameters.SelectMany(p => p
                                 .GroupBy(x => x.DataNode != p.Key ? x.DataNode : x.LinkedDataNode)
                                 .Select(gg =>
                                         {
                                             var currentCandidate = gg.Where(x => x.Year == args.Year).OrderByDescending(x => x.Month).FirstOrDefault();
                                             var previousCandidate = gg.Where(x => x.Year < args.Year).OrderByDescending(x => x.Year).ThenByDescending(x => x.Month).FirstOrDefault();
                                             return (key: p.Key,
                                                     currentPeriod: currentCandidate != null ? currentCandidate : previousCandidate,
                                                     previousPeriod: previousCandidate != null ? previousCandidate : currentCandidate);
                                         })
                                )
        .ToDictionaryGrouped(x => x.key,
                             x => new Dictionary<int, HashSet<InterDataNodeParameter>>{ {CurrentPeriod, x.Select(y => y.currentPeriod).ToHashSet()},
                                                                                       {PreviousPeriod, x.Select(y => y.previousPeriod).ToHashSet()}});
}

## Aoc Step Configuration

In [1]:
public static async Task<IEnumerable<AocConfiguration>> LoadAocStepConfigurationAsync(this IQuerySource querySource, int year, int month)
    => (await querySource.LoadParameterAsync<AocConfiguration>(year, month)).GroupBy(x => (x.AocType, x.Novelty), (k, v) => v.OrderByDescending(x => x.Year).ThenByDescending(x => x.Month).First()); 

In [1]:
public static async Task<Dictionary<AocStep, AocConfiguration>> LoadAocStepConfigurationAsDictionaryAsync(this IQuerySource querySource, int year, int month) => (await querySource.LoadAocStepConfigurationAsync(year, month)).ToDictionary(x => new AocStep(x.AocType, x.Novelty)); 

# Export Configurations

## Main Tab

In [1]:
public static IDocumentBuilder MainTabConfigurationWoScenario<T>(this IDocumentBuilder builder, T args) where T : IfrsPartition
    => builder.WithTable<T>( config => config .AtBeginning() 
        .WithName(Main) 
        .WithSource(source => args.RepeatOnce().AsQueryable()) 
        .WithColumn(x => x.Id, x => x.Delete())
        .WithColumn(x => x.Scenario, x => x.Delete()));

In [1]:
public static IDocumentBuilder MainTabConfiguration<T>(this IDocumentBuilder builder, T args) where T : IfrsPartition
    => builder.WithTable<T>( config => config .AtBeginning() 
        .WithName(Main) 
        .WithSource(source => args.RepeatOnce().AsQueryable()) 
        .WithColumn(x => x.Id, x => x.Delete()));

## Dimensions

In [1]:
public static IDocumentBuilder PortfolioConfiguration<T>(this IDocumentBuilder builder, Type DependsOnType = default) where T : Portfolio
    => builder.WithTable<T>(config => { 
        if(DependsOnType != default)
            config = config.DependsOn(DependsOnType);               
        return config .AtBeginning() 
            .WithColumn(x => x.DisplayName, x => x.AtBeginning())
            .WithColumn(x => x.SystemName, x => x.AtBeginning())
            .WithColumn(x => x.Partition, x => x.Delete())
            .WithColumn(x => x.FunctionalCurrency, x => x.Delete());
    });

In [1]:
public static IDocumentBuilder GroupofContractConfiguration<T>(this IDocumentBuilder builder, Type DependsOnType = default) where T : GroupOfContract
    => builder.WithTable<T>(config => { 
        if(DependsOnType != default)
            config = config.DependsOn(DependsOnType);      
        if(typeof(T).Name == nameof(GroupOfInsuranceContract))
            config = config.WithColumn(x => x.Partner, x => x.Delete());
        return config .AtBeginning() 
            .WithColumn(x => x.DisplayName, x => x.AtBeginning())
            .WithColumn(x => x.SystemName, x => x.AtBeginning())
            .WithColumn(x => x.Partition, x => x.Delete())
            .WithColumn(x => x.ContractualCurrency, x => x.Delete())
            .WithColumn(x => x.FunctionalCurrency, x => x.Delete())
            .WithColumn(x => x.LineOfBusiness, x => x.Delete())
            .WithColumn(x => x.OciType, x => x.Delete())
            .WithColumn(x => x.ValuationApproach, x => x.Delete());
    });

## Data Node State

In [1]:
using DocumentFormat.OpenXml.Spreadsheet;

In [1]:
public record HelperState { public string State {get; init;} }

In [1]:
public static IExcelDocumentBuilder DataNodeStateConfiguration (this IExcelDocumentBuilder builder, DataNodeState[] data)
    => builder
        .WithTable<LiabilityType>(x => x.Delete())
        .WithTable<Profitability>(x => x.Delete())
        .WithTable<Portfolio>(x => x.Delete())
        .WithTable<Currency>(x => x.Delete())
        .WithTable<LineOfBusiness>(x => x.Delete())
        .WithTable<ValuationApproach>(x => x.Delete())
        .WithTable<OciType>(x => x.Delete())
        .WithTable<Partner>(x => x.Delete())
        .WithTable<ReportingNode>(x => x.Delete())
        .WithTable<Scenario>(x => x.Delete())
        .WithTable<DataNodeState>(config => config       
            .AtBeginning() 
            .WithSource(source => data.AsQueryable())
            .WithColumn(x => x.Partition, x => x.Delete())
            .WithColumn(x => x.Month, x => x.Delete())
            .WithColumn(x => x.Year, x => x.Delete())
            .WithColumn(x => x.Id, x => x.Delete())
            .WithColumn(x => x.State, y => y.WithDataValidation(z => z.WithReferenceTo<HelperState, string>(t => t.State)))
        );

In [1]:
public static IExcelDocumentBuilder StateEnumConfiguration (this IExcelDocumentBuilder builder)
{
    var helperState = new[] {new HelperState {State = "Active"}, new HelperState {State = "Inactive"} }; 
    return builder.WithTable<HelperState>( config => config .WithSheetVisibility(SheetStateValues.Hidden)
                //.WithColumn(x => x.State, z => z.WithNamedRange(y => y.WithName("HelperState_State")))
                .WithColumn(x => x.State, z => z.WithDefaultNamedRange())
                .WithSource(source => helperState.AsQueryable()) );
}

## Data Node Parameter

In [1]:
public static IExcelDocumentBuilder DataNodeParameterConfiguration (this IExcelDocumentBuilder builder, Dictionary<string, DataNodeParameter[]> data)
    => builder
        .WithTable<LiabilityType>(x => x.Delete())
        .WithTable<Profitability>(x => x.Delete())
        .WithTable<Portfolio>(x => x.Delete())
        .WithTable<Currency>(x => x.Delete())
        .WithTable<LineOfBusiness>(x => x.Delete())
        .WithTable<ValuationApproach>(x => x.Delete())
        .WithTable<OciType>(x => x.Delete())
        .WithTable<Partner>(x => x.Delete())
        .WithTable<ReportingNode>(x => x.Delete())
        .WithTable<Scenario>(x => x.Delete())
        .WithTable<InterDataNodeParameter>(config => config       
            .AtBeginning() 
            .WithSource(source => data[nameof(InterDataNodeParameter)].Cast<InterDataNodeParameter>().AsQueryable())
            .WithColumn(x => x.Partition, x => x.Delete())
            .WithColumn(x => x.Month, x => x.Delete())
            .WithColumn(x => x.Id, x => x.Delete())
            .WithColumn(x => x.Year, x => x.Delete())
        )
        .WithTable<SingleDataNodeParameter>(config => config       
            .AtBeginning() 
            .WithSource(source => data[nameof(SingleDataNodeParameter)].Cast<SingleDataNodeParameter>().AsQueryable())
            .WithColumn(x => x.DataNode, x => x.WithHeader("DataNode"))
            .WithColumn(x => x.Partition, x => x.Delete())
            .WithColumn(x => x.Month, x => x.Delete())
            .WithColumn(x => x.Year, x => x.Delete())
            .WithColumn(x => x.Year, x => x.Delete())
            .WithColumn(x => x.Id, x => x.Delete())
        );

# Report Configurations

In [1]:
using Systemorph.Vertex.Pivot.Reporting.Builder;

In [1]:
public static ReportBuilder<IfrsVariable,IfrsVariable,IfrsVariable> WithGridOptionsForIfrsVariable
(this ReportBuilder<IfrsVariable,IfrsVariable,IfrsVariable> reportBuilder, int reportHeight = 650)
{
    return reportBuilder.WithGridOptions(go => go.WithColumns(cols => cols.Modify("Value", c => c.WithWidth(300)
                                                                          .WithFormat("new Intl.NumberFormat('en',{ minimumFractionDigits:2, maximumFractionDigits:2 }).format(value)")))
                          .WithRows(rows => rows.Where(r => !r.RowGroup.SystemName.EndsWith("NullGroup")).ToList())
                          .WithAutoGroupColumn(c => c.WithWidth(250) with { Pinned = "left" }) with { Height = reportHeight, GroupDefaultExpanded = 2, OnGridReady = null }
                );
}

# Utility Methods Class

This class contains:
- EqualityComparer

In [1]:
public static class Utils {
    public static void EqualityComparer<T> (T[] collection1, T[] collection2) 
    {
        collection1.Length.Should().Be(collection2.Length);
        var type = typeof(T);
        var properties = type.GetProperties().Where(p => p.Name != "Id").ToArray();
        if(properties.Count() == 0) {
            bool isEqual = Enumerable.SequenceEqual(collection1, collection2);
            isEqual.Should().Be(true);
        }
        foreach(var item1 in collection1) {
            var item2 = collection2.Where(x => 
                properties.All(prop => {
                    var propType = prop.PropertyType;
                    var val = prop.GetValue(item1);
                    var otherVal = prop.GetValue(x);
                    if(val == null && otherVal == null) return true;
                    else if((val != null && otherVal == null) || (val == null && otherVal != null)) return false;
                    else return Convert.ChangeType(otherVal, propType).Equals( Convert.ChangeType(val, propType) );
                })
            );
            item2.Count().Should().NotBe(0);
        }
    }
}

In [1]:
// TODO: find a better arrangement to these methods
static var NumericalPrecisionEqualityChecker = 1.0E-10;

static bool CheckEquality(this double[] arr1, double[] arr2) 
{
    if(arr1.Length != arr2.Length) return false;
    for(int i = 0; i < arr1.Length; i++) {
        double d1 = arr1[i];
        double d2 = arr2[i];
        if(Math.Abs(d1) < NumericalPrecisionEqualityChecker && Math.Abs(d1) < NumericalPrecisionEqualityChecker) continue;
        if(Math.Abs( (d1 - d2) / d1 ) > NumericalPrecisionEqualityChecker) return false;
    }
    return true;
}

static bool CheckEquality(this IEnumerable<double> arr1, double[] arr2) => CheckEquality(arr1.ToArray(), arr2);
static bool CheckEquality(this double[] arr1, IEnumerable<double> arr2) => CheckEquality(arr1, arr2.ToArray());
static bool CheckEquality(this IEnumerable<double> arr1, IEnumerable<double> arr2) => CheckEquality(arr1.ToArray(), arr2.ToArray());
static bool CheckEquality(this double d1, double d2) => CheckEquality(d1.RepeatOnce(), d2.RepeatOnce());

static bool CheckEquality(this double? d1, double? d2) 
{
    if(d1 == null && d2 == null) return true;
    else return CheckEquality((double)d1, (double)d2);
}


## ToIdentityString

In [1]:
using System.Text;
public static string ToIdentityString<T>(this T v)
   where T : class
{
  StringBuilder sb = new StringBuilder();
  var propertyInfos = v.GetType()
                       .GetProperties()
                       .Where(x => Attribute.IsDefined(x, typeof(IdentityPropertyAttribute)))
                       .OrderByDescending(x => x.PropertyType.Name).ThenByDescending(x => x.Name)
                       .Select(x => sb.Append(x.Name).Append(":").Append(v.GetType().GetProperty(x.Name)?.GetValue(v, null)).Append(", "));

return propertyInfos.Select(p => p.ToString()).ToArray().Last();
}

## GetPreviousIdentities

In [1]:
public static Dictionary<AocStep,IEnumerable<AocStep>> GetPreviousIdentities(IEnumerable<AocStep> identities)
{
    var bopNovelties = identities.Where(id => id.AocType == AocTypes.BOP).Select(id => id.Novelty);
    var previousStep = (new string[]{Novelties.N,Novelties.I,Novelties.C})
                                        .ToDictionary(n => n, n => bopNovelties.Contains(n) ? new AocStep(AocTypes.BOP,n) : null);
    return identities.Where(id =>  id.AocType != AocTypes.BOP)
                                .ToDictionary(x => x, x => {var ret = x.AocType == AocTypes.CL ? previousStep.Where(kvp => kvp.Value != null).Select(kvp => kvp.Value).ToArray() 
                                                                                               : previousStep[x.Novelty].RepeatOnce();
                                                            previousStep[x.Novelty] = new AocStep(x.AocType, x.Novelty);
                                                            return ret;});
}

## GetReferenceAocStepForCalculated

In [1]:
public static AocStep GetReferenceAocStepForCalculated(this IEnumerable<AocStep> identities, Dictionary<AocStep, AocConfiguration> aocConfigurationByAocStep, AocStep identityAocStep)
{
    return identities.LastOrDefault(aocStep => aocConfigurationByAocStep[aocStep].DataType != DataType.Calculated
                                            && aocConfigurationByAocStep[aocStep].DataType != DataType.CalculatedTelescopic
                                            && aocConfigurationByAocStep[aocStep].Order < aocConfigurationByAocStep[identityAocStep].Order
                                            && aocStep.Novelty == identityAocStep.Novelty) 
        ?? new AocStep(default, default);
}

# Change Sign Rules

In [1]:
public static int GetSign((string AocType, string AmountType, string EstimateType, bool IsReinsurance) variable, Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache)
{
    var isActual = (new string[] { EstimateTypes.AA, EstimateTypes.OA, EstimateTypes.A }).Contains(variable.EstimateType) ? -1 : 1;
    return isActual;
}

# EstimateType And AmountType

In [1]:
public static Dictionary<string, HashSet<string>> GetAmountTypesByEstimateType(Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache)
{
    return new Dictionary<string, HashSet<string>>(){
        {EstimateTypes.RA, new string[]{}.ToHashSet()},
        {EstimateTypes.C, new string[]{}.ToHashSet()},
        {EstimateTypes.L, new string[]{}.ToHashSet()},
        {EstimateTypes.LR, new string[]{}.ToHashSet()},
   };
}

# Technical Margin EstimateType

In [1]:
public static HashSet<string> GetTechnicalMarginEstimateType()
{
    return new []{EstimateTypes.C, EstimateTypes.L, EstimateTypes.LR, }.ToHashSet();
}

# Discount and Cumulate

In [1]:
public static double[] ComputeDiscountAndCumulate(this double[] nominalValues, double[] monthlyDiscounting, PeriodType periodType) 
{ 
    if(nominalValues == null) return Enumerable.Empty<double>().ToArray();
    
    var ret = new double[nominalValues.Length];
    
    if(periodType == PeriodType.BeginningOfPeriod)
    {
        for (var i = nominalValues.Length - 1; i >= 0; i--)
                ret[i] = nominalValues[i] + GetElementOrDefault(ret, i + 1) * GetElementOrDefault(monthlyDiscounting, i/12);
        return ret;
    }
    
    for (var i = nominalValues.Length - 1; i >= 0; i--)
                ret[i] = ( nominalValues[i] + GetElementOrDefault(ret, i + 1) ) * GetElementOrDefault(monthlyDiscounting, i/12);
    
    return ret;
}

In [1]:
public static double[] ComputeDiscountAndCumulateWithCreditDefaultRisk(this double[] nominalValues, double[] monthlyDiscounting, double nonPerformanceRiskRate) //Is it correct that NonPerformanceRiskRate is a double? Should it be an array that takes as input tau/t?
{ 
    return Enumerable.Range(0, nominalValues.Length)
                     .Select( t => Enumerable.Range(t, nominalValues.Length-t)
                                             .Select( tau => nominalValues[tau] * Math.Pow(GetElementOrDefault(monthlyDiscounting, t/12), tau-t+1) * (Math.Exp(-nonPerformanceRiskRate*(tau-t)) - 1) )
                                             .Sum() )
                     .ToArray();
}

# Import Configuration

## Data Nodes

In [1]:
public GroupOfInsuranceContract ExtendGroupOfContract(GroupOfInsuranceContract gic, IDataRow datarow) => gic;

In [1]:
public GroupOfReinsuranceContract ExtendGroupOfContract(GroupOfReinsuranceContract gric, IDataRow datarow) => gric;